# Graph Convolutional Network in Recommendation System

# Importing Libraries

In [1]:
import numpy as np
%matplotlib inline
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras import backend as K

# 

# Reading the Dataset

In [2]:
df = pd.read_csv('data/Dataset.csv')
series1 = df['user_id']
series2 = df['item_id']
series3 = df['rating']

train_data = list(zip(series1, series2, series3))
train_labels = series3.tolist()

val_data = train_data[60001:72001]
val_labels = train_labels[60001:72001]

# 

# Building a GCN-based recommendation model
`The GCN model is implemented as a subclass of the TensorFlow Keras Model API, and it takes as inputs the user and item indices for a set of ratings. The model then learns embeddings for users and items using two embedding layers. These embeddings are concatenated and passed through a GCN layer to incorporate the graph structure of the user-item interactions.The GCN model is used because it can effectively capture the underlying relationships and interactions between users and items in a graph structure.`

In [166]:
class GraphConvolution(layers.Layer):
    def __init__(self, output_dim):
        super(GraphConvolution, self).__init__()
        self.output_dim = output_dim

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs, adj_matrix):
        # Compute the GCN layer output
        support = tf.matmul(inputs, self.kernel)
        output = tf.matmul(adj_matrix, support)
        return output


class GCNRecommendationModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim, num_hidden_units):
        super(GCNRecommendationModel, self).__init__()
        self.user_embedding = layers.Embedding(num_users, embedding_dim,
                                                embeddings_initializer='he_normal')
        self.item_embedding = layers.Embedding(num_items, embedding_dim,
                                                embeddings_initializer='he_normal')
        self.gcn_layer = GraphConvolution(num_hidden_units)
        self.dropout = layers.Dropout(0.5)
        self.output_layer = layers.Dense(1, activation='relu')

    def call(self, inputs):
        # Extract user and item indices from the inputs
        user_indices = inputs[:, 0]
        item_indices = inputs[:, 1]

        # Embed the users and items
        user_embedding = self.user_embedding(user_indices)
        item_embedding = self.item_embedding(item_indices)

        # Concatenate the user and item embeddings
        concatenated = tf.concat([user_embedding, item_embedding], axis=1)

        # Compute the GCN layer output
        adj_matrix = tf.eye(tf.shape(concatenated)[0])
        gcn_output = self.gcn_layer(concatenated, adj_matrix)

        # Apply dropout and output layer
        output = self.dropout(gcn_output)
        output = self.output_layer(output)

        return output
    
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

model = GCNRecommendationModel(5000, 7000, embedding_dim=16, num_hidden_units=32)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[recall_m])
model.fit(train_data, train_labels, epochs=30, batch_size=64, validation_data=(val_data, val_labels))

Epoch 1/30
1563/1563 [==============================] - 13s 5ms/step - loss: 2.1883 - recall_m: 0.9654 - val_loss: 0.9403 - val_recall_m: 0.9998
Epoch 2/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2335 - recall_m: 0.9996 - val_loss: 0.9086 - val_recall_m: 0.9999
Epoch 3/30
1563/1563 [==============================] - 10s 7ms/step - loss: 1.1069 - recall_m: 0.9997 - val_loss: 0.8636 - val_recall_m: 0.9999
Epoch 4/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0063 - recall_m: 0.9999 - val_loss: 0.8552 - val_recall_m: 0.9999
Epoch 5/30
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9499 - recall_m: 0.9999 - val_loss: 0.8535 - val_recall_m: 1.0000
Epoch 6/30
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9196 - recall_m: 0.9999 - val_loss: 0.8438 - val_recall_m: 1.0000
Epoch 7/30
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9056 - recall_m: 0.9998 - val_loss: 0.8457 - val_recall

# 

# Saving the Model

In [ ]:
model.save('my_gcn_model')

# 

# Merging Item-based Collaborative Filtering with GCN!
`By combining Collaborative Filtering and GCN, we can potentially improve the performance of recommendation systems. The Collaborative Filtering can capture the similarities between users or items based on their ratings, while GCN can capture the latent representation of users and items based on the graph structure and feature information. The combination of the two can potentially overcome the limitations of each individual method and provide better recommendations.`

In [9]:
rating = pd.read_csv("data/Dataset.csv")
movie = pd.read_csv("data/Movie_Id_Titles.csv")
df = pd.merge(movie, rating, on='item_id')

# Average Rating and Number of Ratings + Pivot Table!
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings['rating_numbers'] = pd.DataFrame(df.groupby('title')['rating'].count())
movieRate = df.pivot_table(index='user_id', columns='title', values='rating')
movieRate.fillna(0, inplace=True)

# Recommendation system
def recommendMovies(name , min_rating_count = 50):
    user_rating = movieRate[name]
    similar_movies = movieRate.corrwith(user_rating)
    corr_movies = pd.DataFrame(similar_movies, columns=['Correlation'])
    corr_movies.dropna(inplace=True)
    corr_movies = corr_movies.join(ratings['rating_numbers'], how='left', lsuffix='_left', rsuffix='_right')
    final = corr_movies[corr_movies['rating_numbers']>min_rating_count].sort_values('Correlation', ascending=False)
    dfff = final.merge(df,how='left',on='title')
    dfff = dfff.groupby(['title', 'Correlation', 'rating_numbers', 'item_id']).size().to_frame()[0].to_frame().reset_index()
    dfff = dfff[['title','Correlation','rating_numbers','item_id']].sort_values(by=['Correlation'], ascending=False)
    return dfff.head(20)


def merged(fav_movie):
    df = recommendMovies(fav_movie)
    ser1 = df['title']
    ser2 = df['item_id']

    def predict(user_id):
        prediction = []
        for x in (ser2):
            a = model.predict([(user_id, x)])
            prediction.append(a)
        return prediction

    top = predict(0)

    # combine the Series and list into a DataFrame
    df = pd.DataFrame({'Title': ser1, 'Your Predicted Rating': top})
    df = df.reset_index(drop=True)
    df = df.sort_values(by=['Your Predicted Rating'], ascending=False)
    return df

# 

In [156]:
merged('Game, The (1997)')

1/1 [==============================] - 0s 72ms/step


,Title,Your Predicted Rating
16,Good Will Hunting (1997),[[4.1326284]]
10,L.A. Confidential (1997),[[4.060441]]
12,Titanic (1997),[[4.035684]]
11,Contact (1997),[[3.646411]]
5,Air Force One (1997),[[3.511013]]
0,"Game, The (1997)",[[3.46806]]
4,"Devil's Advocate, The (1997)",[[3.3981512]]
3,"Edge, The (1997)",[[3.3469694]]
6,Kiss the Girls (1997),[[3.3126547]]
14,Cop Land (1997),[[3.3033464]]


# 